Get the embeddings

In [34]:
import openai

client = openai.Client(
    api_key="a",
    base_url="http://83.97.79.137:2003/v1",
)

In [ ]:
# Test the model
client.chat.completions.create(
    model="llama3",
    messages=[
        {
            "role": "user",
            "content": "What is the capital of France?"
        }
    ],
    temperature=0.7,
)

ChatCompletion(id='chatcmpl-123', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1747838544, model='llama3', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=8, prompt_tokens=17, total_tokens=25, completion_tokens_details=None, prompt_tokens_details=None))

In [36]:
client.models.list()

SyncPage[Model](data=[Model(id='mxbai-embed-large:latest', created=1747838353, object='model', owned_by='library'), Model(id='nomic-embed-text:latest', created=1747838331, object='model', owned_by='library'), Model(id='phi4:latest', created=1747827999, object='model', owned_by='library'), Model(id='mixtral:latest', created=1721056010, object='model', owned_by='library'), Model(id='llama3:latest', created=1718724612, object='model', owned_by='library'), Model(id='orca-mini:latest', created=1718724154, object='model', owned_by='library'), Model(id='tinydolphin:latest', created=1718724109, object='model', owned_by='library'), Model(id='tinyllama:latest', created=1718724089, object='model', owned_by='library')], object='list')

In [48]:
res = client.embeddings.create(
    model="all-minilm", #nomic-embed-text", # or mxbai-embed-large
    input="What is the capital of France?"
)

from pprint import pprint
print(pprint(vars(res)))

emb = res.data[0].embedding
len(emb)

{'_request_id': None,
 'data': [Embedding(embedding=[0.08194343, 0.03607655, -0.003874133, -0.0049165883, 0.025664477, -0.057291515, 0.012268374, 0.004728984, 0.035046183, -0.022493396, -0.008033797, -0.10939781, 0.022706956, -0.029179031, -0.04339954, -0.12021584, -0.0007572556, -0.018039843, 0.056222174, 0.003076909, 0.002215925, -0.016780019, 0.063559785, -0.023615263, 0.031379506, -0.034954973, -0.020493886, -0.0030144576, -0.011179857, -0.036107894, 0.05402573, -0.03667546, -0.024982637, -0.038191732, -0.049713403, -0.0151410485, 0.021298718, -0.012793706, 0.07673564, 0.044328265, -0.010962389, -0.029735124, -0.016768651, -0.024673292, 0.008055565, 0.043596547, 0.0071265195, 0.07545383, 0.03282925, -0.06207867, 0.066843115, 0.027075859, -0.04568141, -0.031460077, -0.031150082, 0.09173822, -0.001934343, -0.011360105, 0.036509953, 0.056944475, 0.00233462, -0.037856925, -0.015390687, 0.052394338, 0.060355876, -0.016542653, 0.0090397, -0.0066712443, -0.10618845, 0.001617463, -0.048396

384

In [ ]:
%pip install -q chromadb

ERROR: Operation cancelled by user
^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
import chromadb
from chromadb.config import Settings

# Initialize the ChromaDB client
client = chromadb.PersistentClient(path="db/chromadb")

c = client.get_or_create_collection(name="my_collection")

In [61]:
c.add(
    embeddings=[emb],
    documents=["This is a test document."],
    metadatas=[{"source": "test"}],
    ids=["doc2"]
)

In [57]:
c.delete(ids=["doc1"])

In [59]:
c.get()

{'ids': ['doc1'],
 'embeddings': None,
 'documents': ['This is a test document.'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [{'source': 'test'}]}

In [62]:
c.query(
    query_texts=["france"],
    n_results=10,
    #where={"your_property": True}
)

{'ids': [['doc1', 'doc2']],
 'embeddings': None,
 'documents': [['This is a test document.', 'This is a test document.']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[{'source': 'test'}, {'source': 'test'}]],
 'distances': [[0.7035771012306213, 0.7035771012306213]]}

In [66]:
c.get(
    ids=["doc2"],
    include=["documents", "embeddings", "metadatas"],
    #where={"your_property": True}
)

{'ids': ['doc2'],
 'embeddings': array([[ 8.19434300e-02,  3.60765494e-02, -3.87413311e-03,
         -4.91658831e-03,  2.56644767e-02, -5.72915152e-02,
          1.22683737e-02,  4.72898409e-03,  3.50461826e-02,
         -2.24933960e-02, -8.03379714e-03, -1.09397814e-01,
          2.27069557e-02, -2.91790310e-02, -4.33995388e-02,
         -1.20215841e-01, -7.57255591e-04, -1.80398431e-02,
          5.62221743e-02,  3.07690911e-03,  2.21592491e-03,
         -1.67800188e-02,  6.35597855e-02, -2.36152634e-02,
          3.13795060e-02, -3.49549726e-02, -2.04938855e-02,
         -3.01445764e-03, -1.11798570e-02, -3.61078940e-02,
          5.40257283e-02, -3.66754606e-02, -2.49826368e-02,
         -3.81917320e-02, -4.97134030e-02, -1.51410485e-02,
          2.12987177e-02, -1.27937058e-02,  7.67356381e-02,
          4.43282649e-02, -1.09623894e-02, -2.97351237e-02,
         -1.67686511e-02, -2.46732924e-02,  8.05556495e-03,
          4.35965471e-02,  7.12651946e-03,  7.54538327e-02,
        